In [1]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, root_mean_squared_error , mean_squared_error

logical_cores = os.cpu_count()
print(f"Number of logical CPU cores: {logical_cores}")

num_workers = max(1, logical_cores // 2)
print(f"Number of workers set to: {num_workers}")

def is_gpu_available():
    try:
        return torch.cuda.is_available()
    except ImportError:
        return False

gpu_available = is_gpu_available()
print(f"GPU available: {gpu_available}")

Number of logical CPU cores: 16
Number of workers set to: 8
GPU available: True


In [2]:
os.makedirs('../models/pytorch/conv1d-classification/', exist_ok=True)
os.makedirs('../models/pytorch/conv1d-regression/', exist_ok=True)
os.makedirs('../models/pytorch/lstm-classification/', exist_ok=True)
os.makedirs('../models/pytorch/lstm-regression/', exist_ok=True)
os.makedirs('../models/pytorch/conv1d-classification-lstm-classification/', exist_ok=True)
os.makedirs('../models/pytorch/conv1d-classification-lstm-regression/', exist_ok=True)
os.makedirs('../models/pytorch/conv1d-regression-lstm-classification/', exist_ok=True)
os.makedirs('../models/pytorch/conv1d-regression-lstm-regression/', exist_ok=True)

path = '../data'
ticker_list = []

if os.path.exists(path):
    ticker_list = [os.path.splitext(f)[0] for f in os.listdir(path) if f.endswith('.csv')]


In [3]:
def preprocess_data(df):
    if df.isna().sum().sum() > 0 or df.isin([float('inf'), float('-inf')]).sum().sum() > 0:
        df = df.replace([float('inf'), float('-inf')], float('nan')).dropna()

    df = df.dropna()

    columns_to_drop = [
        'NEXT_DAY_CLOSEPRICE', 'DAILY_CLOSEPRICE_CHANGE', 'CLOSEPRICE_DIRECTION',
        'DAILY_MIDPRICE', 'NEXT_DAY_MIDPRICE', 'DAILY_MIDPRICE_CHANGE', 'MIDPRICE_DIRECTION', 'Date'
    ]
    X = df.drop(columns=columns_to_drop)
    y_classifier = (df['DAILY_CLOSEPRICE_CHANGE'] > 0).astype(int)
    y_regressor = df['DAILY_CLOSEPRICE_CHANGE']

    return X, y_classifier, y_regressor

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, l2_lambda=0.01):
        super(ResidualBlock, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding='same')
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, stride=1, padding='same')
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
        nn.init.kaiming_normal_(self.conv1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv2.weight, nonlinearity='relu')
        nn.init.zeros_(self.conv1.bias)
        nn.init.zeros_(self.conv2.bias)
        
        self.l2_lambda = l2_lambda

        if in_channels != out_channels:
            self.residual_conv = nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1)
        else:
            self.residual_conv = nn.Identity()

    def forward(self, x):
        residual = self.residual_conv(x)
        print(f'ResidualBlock - Input shape: {x.shape}')
        out = self.conv1(x)
        print(f'ResidualBlock - After conv1: {out.shape}')
        out = self.bn1(out)
        print(f'ResidualBlock - After bn1: {out.shape}')
        out = self.relu(out)
        print(f'ResidualBlock - After relu1: {out.shape}')
        out = self.conv2(out)
        print(f'ResidualBlock - After conv2: {out.shape}')
        out = self.bn2(out)
        print(f'ResidualBlock - After bn2: {out.shape}')
        out += residual
        print(f'ResidualBlock - After adding residual: {out.shape}')
        out = self.relu(out)
        print(f'ResidualBlock - After relu2: {out.shape}')
        
        return out

class Conv1DModel(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, num_blocks=1, l2_lambda=0.01, classification=True):
        super(Conv1DModel, self).__init__()
        self.blocks = nn.Sequential(
            *[ResidualBlock(in_channels, out_channels, kernel_size, l2_lambda=l2_lambda) for _ in range(num_blocks)]
        )
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)  # Global average pooling for 1D
        self.fc = nn.Linear(out_channels, 2 if classification else 1)
        self.classification = classification

    def forward(self, x):
        print(f'Conv1DModel - Input shape: {x.shape}')
        out = self.blocks(x)
        print(f'Conv1DModel - After residual blocks: {out.shape}')
        out = self.global_avg_pool(out)
        print(f'Conv1DModel - After global average pooling: {out.shape}')
        out = out.view(out.size(0), -1)  # Flatten the tensor
        print(f'Conv1DModel - After flattening: {out.shape}')
        out = self.fc(out)
        print(f'Conv1DModel - After fully connected layer: {out.shape}')
        if self.classification:
            out = F.log_softmax(out, dim=1)
            print(f'Conv1DModel - After log_softmax: {out.shape}')
        return out

In [5]:
def process_conv1d(X, y, classification, gpu_available, ticker):
    device = torch.device('cuda' if gpu_available and torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    
    # Convert DataFrame to numpy array
    X = X.to_numpy()
    y = y.to_numpy()
    
    # Reshape X for Conv1D
    NUM_CHANNELS = 1
    X = X.reshape((X.shape[0], NUM_CHANNELS, -1))  # Reshape for Conv1D: (batch_size, num_channels, sequence_length)
    print(f'process_conv1d - Input shape after reshaping: {X.shape}')
    
    # Split data into training and validation sets
    TEST_SIZE = 0.2
    RANDOM_STATE = 42
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
    print(f'process_conv1d - Training data shape: {X_train.shape}, Validation data shape: {X_val.shape}')
    
    def conv1d_objective(trial):
        in_channels = X_train.shape[1]  # Ensure this matches the reshaped input
        out_channels = trial.suggest_int('out_channels', 16, 128)
        kernel_size = trial.suggest_int('kernel_size', 3, 7)
        num_blocks = trial.suggest_int('num_blocks', 1, 10)
        l2_lambda = trial.suggest_float('l2_lambda', 1e-5, 1e-2)
        
        model = Conv1DModel(in_channels, out_channels, kernel_size, num_blocks, l2_lambda, classification).to(device)
        print(f'conv1d_objective - Model initialized with in_channels={in_channels}, out_channels={out_channels}, kernel_size={kernel_size}, num_blocks={num_blocks}')
        
        optimizer = optim.Adam(model.parameters(), lr=trial.suggest_float('lr', 1e-5, 1e-2), weight_decay=l2_lambda)
        criterion = nn.CrossEntropyLoss() if classification else nn.MSELoss()
        
        input_train = torch.tensor(X_train, dtype=torch.float32).to(device)
        target_train = torch.tensor(y_train, dtype=torch.long if classification else torch.float32).to(device)
        
        model.train()
        EPOCHS = 10
        for epoch in range(EPOCHS):
            optimizer.zero_grad()
            output = model(input_train)
            print(f'conv1d_objective - Epoch {epoch}: Output shape: {output.shape}')
            loss = criterion(output, target_train)
            loss.backward()
            optimizer.step()
        
        # Validation
        model.eval()
        input_val = torch.tensor(X_val, dtype=torch.float32).to(device)
        target_val = torch.tensor(y_val, dtype=torch.long if classification else torch.float32).to(device)
        with torch.no_grad():
            val_output = model(input_val)
            val_loss = criterion(val_output, target_val)
            if classification:
                val_accuracy = (val_output.argmax(dim=1) == target_val).float().mean().item()
                print(f'conv1d_objective - Trial {trial.number}: Validation Accuracy = {val_accuracy:.4f}, Validation Loss = {val_loss.item():.4f}')
                return 1 - val_accuracy
            else:
                val_mse = val_loss.item()
                print(f'conv1d_objective - Trial {trial.number}: Validation MSE = {val_mse:.4f}, Validation Loss = {val_loss.item():.4f}')
                return val_mse
    
    study = optuna.create_study(direction='minimize')
    study.optimize(conv1d_objective, n_trials=100)
    
    best_model = Conv1DModel(X.shape[1], study.best_params['out_channels'], study.best_params['kernel_size'], study.best_params['num_blocks'], study.best_params['l2_lambda'], classification).to(device)
    
    # Save the best model
    model_type = 'classification' if classification else 'regression'
    torch.save(best_model.state_dict(), f'../models/pytorch/conv1d-{model_type}/{ticker}.pth')
    
    # Print out metrics for the best model
    best_model.eval()
    input_val = torch.tensor(X_val, dtype=torch.float32).to(device)
    target_val = torch.tensor(y_val, dtype=torch.long if classification else torch.float32).to(device)
    with torch.no_grad():
        val_output = best_model(input_val)
        val_loss = criterion(val_output, target_val)
        val_accuracy = (val_output.argmax(dim=1) == target_val).float().mean().item()
        val_mse =val_mse = val_loss.item()
        print(f'Validation Accuracy: {val_accuracy:.4f}')
        print(f'Validation MSE: {val_mse:.4f}')
    

In [6]:
for ticker in ticker_list:
    dataframe = pd.read_csv(f"../data/{ticker}.csv")
    X, y_classifier, y_regressor = preprocess_data(dataframe)
    process_conv1d(X, y_classifier, True, gpu_available, ticker)
    #process_conv1dlstm(X, y_classifier, y_regressor, True, False, gpu_available, ticker)

[I 2024-09-04 12:14:31,034] A new study created in memory with name: no-name-44a4cc74-5690-46e3-8eb3-26e98fff73b2


Using device: cuda
process_conv1d - Input shape after reshaping: (163, 1, 100)
process_conv1d - Training data shape: (130, 1, 100), Validation data shape: (33, 1, 100)
conv1d_objective - Model initialized with in_channels=1, out_channels=105, kernel_size=4, num_blocks=6
Conv1DModel - Input shape: torch.Size([130, 1, 100])


C:\Users\ng_mi\Anaconda\envs\pytorch-gpu\Lib\site-packages\torch\nn\modules\conv.py:304: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Convolution.cpp:1032.)
  return F.conv1d(input, weight, bias, self.stride,
[W 2024-09-04 12:14:32,601] Trial 0 failed with parameters: {'out_channels': 105, 'kernel_size': 4, 'num_blocks': 6, 'l2_lambda': 0.0055368679208314285, 'lr': 0.0030196163140850814} because of the following error: RuntimeError('Given groups=1, weight of size [105, 1, 1], expected input[130, 105, 100] to have 1 channels, but got 105 channels instead').
Traceback (most recent call last):
  File "C:\Users\ng_mi\Anaconda\envs\pytorch-gpu\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\ng_mi\AppData\Local\Temp\ipykerne

ResidualBlock - Input shape: torch.Size([130, 1, 100])
ResidualBlock - After conv1: torch.Size([130, 105, 100])
ResidualBlock - After bn1: torch.Size([130, 105, 100])
ResidualBlock - After relu1: torch.Size([130, 105, 100])
ResidualBlock - After conv2: torch.Size([130, 105, 100])
ResidualBlock - After bn2: torch.Size([130, 105, 100])
ResidualBlock - After adding residual: torch.Size([130, 105, 100])
ResidualBlock - After relu2: torch.Size([130, 105, 100])


RuntimeError: Given groups=1, weight of size [105, 1, 1], expected input[130, 105, 100] to have 1 channels, but got 105 channels instead

In [6]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, sequence_length, classification=True):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sequence_length = sequence_length
        self.classification = classification

    def forward(self, x):
        h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        if self.classification:
            out = F.log_softmax(out, dim=1)
        return out

def process_lstm(X, y, classification, gpu_available, ticker):
    device = torch.device('cuda' if gpu_available and torch.cuda.is_available() else 'cpu')

    # Convert DataFrame to numpy array
    X = X.to_numpy()
    y = y.to_numpy()
  
    # Reshape X for LSTM
    X = X.reshape((X.shape[0], -1, 1))  # Reshape for LSTM: (batch_size, sequence_length, num_features)
    
    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    input_val = torch.tensor(X_val, dtype=torch.float32).to(device)
    target_val = torch.tensor(y_val, dtype=torch.long if classification else torch.float32).to(device)
    
    def lstm_objective(trial):
        input_size = X.shape[2]
        hidden_size = trial.suggest_int('hidden_size', 16, 128)
        num_layers = trial.suggest_int('num_layers', 1, 3)
        sequence_length = X.shape[1]
        
        model = LSTMModel(input_size, hidden_size, num_layers, 2 if classification else 1, sequence_length, classification).to(device)
        
        optimizer = optim.Adam(model.parameters(), lr=trial.suggest_float('lr', 1e-5, 1e-2))
        criterion = nn.CrossEntropyLoss() if classification else nn.MSELoss()
        
        input_train = torch.tensor(X_train, dtype=torch.float32).to(device)
        target_train = torch.tensor(y_train, dtype=torch.long if classification else torch.float32).to(device)
        
        model.train()
        for epoch in range(10):
            optimizer.zero_grad()
            output = model(input_train)
            loss = criterion(output, target_train)
            loss.backward()
            optimizer.step()
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_output = model(input_val)
            val_loss = criterion(val_output, target_val)
            val_accuracy = (val_output.argmax(dim=1) == target_val).float().mean().item()
            val_mse =val_mse = val_loss.item()
            print(f'Trial {trial.number}: Validation Accuracy = {val_accuracy:.4f}, Validation MSE = {val_mse:.4f}, Validation Loss = {val_loss.item():.4f}')
            return 1 - accuracy if classification else mse
    
    study = optuna.create_study(direction='minimize')
    study.optimize(lstm_objective, n_trials=100)
    
    best_model = LSTMModel(X.shape[2], study.best_params['hidden_size'], study.best_params['num_layers'], 2 if classification else 1, X.shape[1], classification).to(device)
    
    # Save the best model
    model_type = 'classification' if classification else 'regression'
    torch.save(best_model.state_dict(), f'../models/pytorch/lstm-{model_type}/{ticker}.pth')
    
    # Print out metrics for the best model
    best_model.eval()
    with torch.no_grad():
        val_output = best_model(input_val)
        val_loss = criterion(val_output, target_val)
        val_accuracy = (val_output.argmax(dim=1) == target_val).float().mean().item()
        val_mse =val_mse = val_loss.item()
        print(f'Validation Accuracy: {val_accuracy:.4f}')
        print(f'Validation MSE: {val_mse:.4f}')
  

In [7]:
class ConvLSTMModel(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, lstm_input_size, lstm_hidden_size, lstm_num_layers, output_size, classification=True):
        super(ConvLSTMModel, self).__init__()
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size)
        self.lstm = nn.LSTM(lstm_input_size, lstm_hidden_size, lstm_num_layers, batch_first=True)
        self.fc = nn.Linear(lstm_hidden_size, output_size)
        self.classification = classification

    def forward(self, x):
        x = self.conv1d(x)
        x = x.permute(0, 2, 1)  # Change shape to (batch_size, sequence_length, num_channels)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x

def process_conv1dlstm(X, y_classifier, y_regressor, conv1d_classification, lstm_classification, gpu_available, ticker):
    device = torch.device('cuda' if gpu_available and torch.cuda.is_available() else 'cpu')
    
    # Convert DataFrame to numpy array
    X = X.to_numpy()
    y_classifier = y_classifier.to_numpy()
    y_regressor = y_regressor.to_numpy()
    
    # Reshape X for ConvLSTM
    X = X.reshape((X.shape[0], 1, X.shape[1]))  # Reshape for Conv1D: (batch_size, num_channels, sequence_length)
    
    # Split data into training and validation sets
    y = y_classifier if conv1d_classification else y_regressor
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    input_val = torch.tensor(X_val, dtype=torch.float32).to(device)
    target_val = torch.tensor(y_val, dtype=torch.long if conv1d_classification else torch.float32).to(device)

    criterion = nn.CrossEntropyLoss() if conv1d_classification else nn.MSELoss()
    
    def conv_lstm_objective(trial):
        in_channels = X.shape[1]
        out_channels = trial.suggest_int('out_channels', 16, 128)
        kernel_size = trial.suggest_int('kernel_size', 3, 7)
        lstm_hidden_size = trial.suggest_int('lstm_hidden_size', 16, 128)
        lstm_num_layers = trial.suggest_int('lstm_num_layers', 1, 3)
        output_size = len(set(y_classifier)) if conv1d_classification else 1  # Number of classes for classification
        
        model = ConvLSTMModel(in_channels, out_channels, kernel_size, out_channels, lstm_hidden_size, lstm_num_layers, output_size, classification=conv1d_classification).to(device)
        
        optimizer = optim.Adam(model.parameters(), lr=trial.suggest_float('lr', 1e-5, 1e-2))
        criterion = nn.CrossEntropyLoss() if conv1d_classification else nn.MSELoss()
        
        input_train = torch.tensor(X_train, dtype=torch.float32).to(device)
        target_train = torch.tensor(y_train, dtype=torch.long if conv1d_classification else torch.float32).to(device)
        
        # Training loop
        model.train()
        for epoch in range(10):
            optimizer.zero_grad()
            output = model(input_train)
            loss = criterion(output, target_train)
            loss.backward()
            optimizer.step()
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_output = model(input_val)
            val_loss = criterion(val_output, target_val)
            val_accuracy = (val_output.argmax(dim=1) == target_val).float().mean().item()
            val_mse =val_mse = val_loss.item()
            print(f'Trial {trial.number}: Validation Accuracy = {val_accuracy:.4f}, Validation MSE = {val_mse:.4f}, Validation Loss = {val_loss.item():.4f}')
            return val_loss.item()
    
    study = optuna.create_study(direction='minimize')
    study.optimize(conv_lstm_objective, n_trials=100)
    
    best_params = study.best_params
    best_model = ConvLSTMModel(X.shape[1], best_params['out_channels'], best_params['kernel_size'], best_params['out_channels'], best_params['lstm_hidden_size'], best_params['lstm_num_layers'], len(set(y_classifier)) if conv1d_classification else 1, classification=conv1d_classification).to(device)
    
    # Save the best model
    conv_model_type = 'classification' if conv1d_classification else 'regression'
    lstm_model_type = 'classification' if lstm_classification else 'regression'
    torch.save(best_model.state_dict(), f'../models/pytorch/conv1d-{conv_model_type}-lstm-{lstm_model_type}/{ticker}.pth')
    
    # Evaluate the best model on the validation set
    best_model.eval()
    with torch.no_grad():
        val_output = best_model(input_val)
        val_loss = criterion(val_output, target_val)
        val_accuracy = (val_output.argmax(dim=1) == target_val).float().mean().item()
        val_mse =val_mse = val_loss.item()
        print(f'Validation Accuracy: {val_accuracy:.4f}')
        print(f'Validation MSE: {val_mse:.4f}')